# Project Draft: Deep Learning-Enhanced SLAM for Office Robotics
## 1. Abstract
### This project aims to develop a deep learning-enhanced Simultaneous Localization and Mapping (SLAM) system on a Raspberry Pi-powered robot. The robot, equipped with a camera module, will autonomously navigate an office environment and generate an interactive 3D map. A custom lightweight Convolutional Neural Network (CNN) will be developed for real-time feature extraction to enhance traditional SLAM algorithms. The final deliverable will include an interactive 3D visualization of the mapped environment, showcasing the system's performance and potential applications.

In [ ]:
import cv2
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Definition of a light weight model that can be deployed on a Raspberry Pi
def create_cnn_model(input_shape=(64, 64, 3)):
    model = keras.Sequential([
        # Convolutional layer 1
        layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(pool_size=(2, 2)),

        # Convolutional layer 2
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),

        # Flatten and fully connected layers
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),

        # Output: Feature vector
        layers.Dense(10, activation='linear')  # 10 feature descriptors
    ])
    
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# Creating the model
cnn_model = create_cnn_model()
cnn_model.summary()

The project will require pulling images from the Raspberry Pi Camera. The following code is structured to assist in grabbing images and preparing for training the model.

In [ ]:
# Create dataset directory
dataset_path = "dataset"
os.makedirs(dataset_path, exist_ok=True)

cap = cv2.VideoCapture(0)  # Access the Pi Camera

count = 0
while count < 100:  # Collect 100 images
    ret, frame = cap.read()
    if not ret:
        break
    
    img_path = os.path.join(dataset_path, f"image_{count}.jpg")
    cv2.imwrite(img_path, frame)
    count += 1
    print(f"Saved {img_path}")

cap.release()
cv2.destroyAllWindows()

Next, the CNN is given access to these images for training.

In [ ]:
# Data preprocessing
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(64, 64),
    batch_size=16,
    class_mode=None,  # Unsupervised
    subset='training'
)

val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(64, 64),
    batch_size=16,
    class_mode=None,
    subset='validation'
)

# Train the model
cnn_model.fit(train_data, epochs=10, validation_data=val_data)